# TD SIEM : Suricata, Filebeat, Elasticsearch

## Objectifs

Ce TD vous permettra de :
- Vérifier le bon fonctionnement de la stack SIEM
- Analyser les données indexées par Filebeat
- Détecter des anomalies sans ML
- Détecter des anomalies avec ML (Isolation Forest)

## Prérequis

1. Démarrer la stack :
```bash
docker-compose -f docker-compose-siem.yml up -d
```

2. Attendre quelques minutes que Suricata génère des logs et que Filebeat les indexe dans Elasticsearch.

In [1]:
# Configuration et connexion à Elasticsearch
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta
import json
import subprocess
import os

# Configuration
ES_HOST = "https://localhost:9200"
ES_USER = "elastic"
ES_PASSWORD = "changeme"  # Modifiez selon votre .env

# Connexion
es = Elasticsearch(
    [ES_HOST],
    basic_auth=(ES_USER, ES_PASSWORD),
    verify_certs=False
)

# Vérification de la connexion
health = es.cluster.health()
print(f"✅ Cluster Elasticsearch: {health['status']} ({health['number_of_nodes']} nœuds)")

✅ Cluster Elasticsearch: green (3 nœuds)


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/elasticsearch/_sync/client/__init__.py:313: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## 1. Vérification de la stack

In [2]:
# Vérification des services Docker
services = ['es01', 'es02', 'es03', 'kibana', 'suricata', 'filebeat']
running = []

for service in services:
    try:
        result = subprocess.run(
            ['docker', 'ps', '--filter', f'name={service}', '--format', '{{.Names}}'],
            capture_output=True, text=True, timeout=5
        )
        if service in result.stdout:
            running.append(service)
            print(f"✅ {service}")
        else:
            print(f"❌ {service}")
    except:
        print(f"❌ {service}")

if len(running) == len(services):
    print(f"\n✅ Tous les services sont démarrés ({len(running)}/{len(services)})")
else:
    print(f"\n⚠️  Services démarrés: {len(running)}/{len(services)}")

✅ es01
✅ es02
✅ es03
✅ kibana
✅ suricata
✅ filebeat

✅ Tous les services sont démarrés (6/6)


## 2. Vérification de l'injection des données

In [4]:
# Recherche des index Suricata
def get_suricata_index():
    """Retourne le nom de l'index Suricata le plus récent"""
    try:
        indices = es.indices.get(index="suricata-*")
        if indices:
            return sorted(indices.keys())[-1]
    except:
        pass
    return "suricata-*"

index_name = get_suricata_index()

# Comptage des documents
try:
    count = es.count(index=index_name)
    print(f"📊 Index: {index_name}")
    print(f"📈 Nombre de documents: {count['count']:,}")
    
    # Exemple de document
    if count['count'] > 0:
        sample = es.search(index=index_name, size=1, query={"match_all": {}})
        if sample['hits']['hits']:
            doc = sample['hits']['hits'][0]['_source']
            print(f"\n📄 Exemple de document:")
            print(f"   Type: {doc.get('event_type', 'N/A')}")
            print(f"   Timestamp: {doc.get('@timestamp', doc.get('timestamp', 'N/A'))}")
            if 'src_ip' in doc:
                print(f"   Source: {doc.get('src_ip')}:{doc.get('src_port', 'N/A')}")
                print(f"   Destination: {doc.get('dest_ip')}:{doc.get('dest_port', 'N/A')}")
            if 'alert' in doc:
                alert = doc['alert']
                print(f"   Alerte: {alert.get('signature', 'N/A')}")
                print(f"   Sévérité: {alert.get('severity', 'N/A')}")
except Exception as e:
    print(f"❌ Erreur: {e}")

📊 Index: .ds-suricata-2026.01.17-2026.01.17-000001
📈 Nombre de documents: 129,374

📄 Exemple de document:
   Type: flow
   Timestamp: 2026-01-17T16:54:19.631Z
   Source: 192.168.65.1:23560
   Destination: 192.168.65.7:2376


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanc

## 3. Détection d'anomalies sans ML

In [5]:
# Détection d'anomalies basiques basées sur des règles
def detect_anomalies_basic():
    """Détecte des anomalies sans ML"""
    anomalies = []
    
    # 1. IPs sources avec beaucoup d'alertes différentes (scan suspect)
    query = {
        "size": 0,
        "query": {"term": {"event_type": "alert"}},
        "aggs": {
            "suspicious_ips": {
                "terms": {"field": "src_ip", "size": 10},
                "aggs": {
                    "unique_signatures": {"cardinality": {"field": "alert.signature_id"}},
                    "unique_dest_ips": {"cardinality": {"field": "dest_ip"}},
                    "total_alerts": {"value_count": {"field": "event_type"}}
                }
            }
        }
    }
    
    result = es.search(index=index_name, body=query)
    
    print("🔍 Détection d'anomalies (règles basiques)\n")
    
    for bucket in result['aggregations']['suspicious_ips']['buckets']:
        ip = bucket['key']
        unique_sigs = bucket['unique_signatures']['value']
        unique_dests = bucket['unique_dest_ips']['value']
        total = bucket['total_alerts']['value']
        
        # Critères d'anomalie
        if unique_sigs > 3 or unique_dests > 5:
            anomalies.append({
                "ip": ip,
                "type": "Scan suspect",
                "signatures": unique_sigs,
                "destinations": unique_dests,
                "total": total
            })
            print(f"🚨 {ip}: {unique_sigs} signatures, {unique_dests} destinations ({total} alertes)")
    
    # 2. Alertes de haute sévérité récentes
    time_threshold = datetime.now() - timedelta(hours=1)
    query_critical = {
        "bool": {
            "must": [
                {"term": {"event_type": "alert"}},
                {"range": {"alert.severity": {"lte": 1}}}
            ],
            "filter": [
                {"range": {"@timestamp": {"gte": time_threshold.isoformat()}}}
            ]
        }
    }
    
    critical = es.search(index=index_name, query=query_critical, size=10)
    if critical['hits']['total']['value'] > 0:
        print(f"\n🔴 Alertes critiques récentes: {critical['hits']['total']['value']}")
        for hit in critical['hits']['hits'][:3]:
            src = hit['_source']
            alert = src.get('alert', {})
            print(f"   - {alert.get('signature', 'N/A')} (Sév: {alert.get('severity', 'N/A')})")
    
    return anomalies

anomalies = detect_anomalies_basic()
if not anomalies:
    print("\n✅ Aucune anomalie détectée avec les règles basiques")

🔍 Détection d'anomalies (règles basiques)


✅ Aucune anomalie détectée avec les règles basiques


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## 4. Détection d'anomalies avec ML (Isolation Forest)

In [6]:
# Préparation des données pour ML
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

def prepare_features():
    """Extrait et prépare les features depuis Elasticsearch"""
    # Agrégation par IP source avec statistiques
    query = {
        "size": 0,
        "query": {"term": {"event_type": "alert"}},
        "aggs": {
            "by_ip": {
                "terms": {"field": "src_ip", "size": 100},
                "aggs": {
                    "avg_severity": {"avg": {"field": "alert.severity"}},
                    "unique_signatures": {"cardinality": {"field": "alert.signature_id"}},
                    "unique_dest_ips": {"cardinality": {"field": "dest_ip"}},
                    "unique_dest_ports": {"cardinality": {"field": "dest_port"}},
                    "total_alerts": {"value_count": {"field": "event_type"}}
                }
            }
        }
    }
    
    result = es.search(index=index_name, body=query)
    
    # Construction du DataFrame
    data = []
    for bucket in result['aggregations']['by_ip']['buckets']:
        data.append({
            'ip': bucket['key'],
            'avg_severity': bucket['avg_severity']['value'] or 0,
            'unique_signatures': bucket['unique_signatures']['value'],
            'unique_dest_ips': bucket['unique_dest_ips']['value'],
            'unique_dest_ports': bucket['unique_dest_ports']['value'],
            'total_alerts': bucket['total_alerts']['value']
        })
    
    if not data:
        print("⚠️  Aucune donnée d'alerte disponible pour l'analyse ML")
        return None, None
    
    df = pd.DataFrame(data)
    return df, df[['avg_severity', 'unique_signatures', 'unique_dest_ips', 'unique_dest_ports', 'total_alerts']]

df, features = prepare_features()

if df is not None and len(df) > 0:
    print(f"📊 {len(df)} IPs analysées")
    print(f"📈 Features: avg_severity, unique_signatures, unique_dest_ips, unique_dest_ports, total_alerts")
else:
    print("❌ Impossible de préparer les données")

⚠️  Aucune donnée d'alerte disponible pour l'analyse ML
❌ Impossible de préparer les données


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
# Application d'Isolation Forest
if df is not None and len(df) > 0:
    # Normalisation
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    predictions = iso_forest.fit_predict(features_scaled)
    
    # Ajout des prédictions au DataFrame
    df['anomaly_score'] = iso_forest.score_samples(features_scaled)
    df['is_anomaly'] = predictions == -1
    
    # Affichage des anomalies détectées
    anomalies_ml = df[df['is_anomaly']].sort_values('anomaly_score')
    
    print(f"\n🔍 Détection d'anomalies avec Isolation Forest")
    print(f"📊 {len(anomalies_ml)} anomalies détectées sur {len(df)} IPs\n")
    
    if len(anomalies_ml) > 0:
        print("🚨 IPs anormales (top 10):")
        for idx, row in anomalies_ml.head(10).iterrows():
            print(f"   {row['ip']}: score={row['anomaly_score']:.2f}, "
                  f"signatures={row['unique_signatures']}, "
                  f"destinations={row['unique_dest_ips']}, "
                  f"alertes={row['total_alerts']}")
    else:
        print("✅ Aucune anomalie détectée par ML")
else:
    print("⚠️  Pas assez de données pour l'analyse ML")

⚠️  Pas assez de données pour l'analyse ML


## Résumé

- ✅ Stack vérifiée et fonctionnelle
- ✅ Données indexées et analysables
- ✅ Détection d'anomalies basiques opérationnelle
- ✅ Détection d'anomalies ML (Isolation Forest) opérationnelle

**Prochaines étapes :**
- Explorer les données dans Kibana
- Affiner les règles de détection
- Ajuster les paramètres ML (contamination, features)